In [11]:
import numpy as np
from fxpmath import Fxp
import matplotlib.pyplot as plt
from FPGAJacobi.Cordic import Cordic

In [12]:
class Jacobi:
    def __init__(self, 
                 n_cordic_iterations : int = 12, 
                 n_word : int = 18, 
                 n_frac : int = 15, 
                 is_signed : bool = True,
                 matrix_dim : int = 12,
                 n_sweeps : int = 1):
                
                 
    
        self._n_word = n_word
        self._n_frac = n_frac
        self._is_signed = is_signed
        self._rotation_cordic = Cordic(mode = "rotation", 
                              n_iterations = n_cordic_iterations, 
                              n_word = self._n_word, 
                              n_frac = self._n_frac, 
                              is_signed = True)
        self._vectoring_cordic = Cordic(mode = "vectoring", 
                              n_iterations = n_cordic_iterations, 
                              n_word = self._n_word, 
                              n_frac = self._n_frac, 
                              is_signed = True)
        self._N = matrix_dim
        self._A = Fxp(np.zeros((self._N, self._N)), n_word = self._n_word, n_frac = self._n_frac, signed = self._is_signed,
                      rounding = "around")
        self._V = Fxp(np.zeros((self._N, self._N)), n_word = self._n_word, n_frac = self._n_frac, signed = self._is_signed,
                      rounding = "around")
        
        self._round_state = np.arange(self._N)
        
        self._n_sweeps = n_sweeps
        
    
    def get_A(self, A : Fxp):
        self._A.equal(A)
    
    def init_V(self):
        self._V.set_val(np.eye(self._N))
        
    def init_round_state(self):
        self._round_state = np.arange(self._N)
        
    def update_matrices(self, i,j):
        
        zero = Fxp(0, n_word = self._n_word, n_frac = self._n_frac, signed = self._is_signed, rounding = "around")
        
        Aij2 = self._A[i,j] << 1
        pi_to_add = 0*zero
        sign = 0*zero
        
        #hadle |2theta| > 90 degrees 
        if (self._A[j,j] > self._A[i,i]):
            Ajj_ii= self._A[j,j] - self._A[i,i]     
        else:
            Ajj_ii= -self._A[j,j] + self._A[i,i]
            Aij2 = - Aij2
            pi_to_add.set_val(np.pi)
        
        if (Aij2 > 0):
            sign.set_val(-1)
        else:
            sign.set_val(1)
        
        x, y, theta = self._vectoring_cordic.run(Ajj_ii, Aij2, zero)
        theta = (theta + sign*pi_to_add) >> 1
        
        #diagonal
        temp_ii_i, temp_ii_j, _ = self._rotation_cordic.run(self._A[i,i], self._A[i,j], theta)
        temp_jj_i, temp_jj_j, _ = self._rotation_cordic.run(self._A[i,j], self._A[j,j], theta)
        self._A[i,i], _, _ = self._rotation_cordic.run(temp_ii_i, temp_jj_i, theta)
        _, self._A[j,j], _ = self._rotation_cordic.run(temp_ii_j, temp_jj_j, theta)
          
        #not diagonal i,j
        self._A[i,j] = self._A[j,i] = 0
        
        #not diagonal != i,j
        for k in range(self._N):
            if (k != i and k != j):
                self._A[k,i], self._A[k,j], _ = self._rotation_cordic.run(self._A[k,i], self._A[k,j], theta)
                self._A[i,k] = self._A[k,i]
                self._A[j,k] = self._A[k,j]
    
        #Update V matrix
        for k in range(self._N):
            self._V[k,i], self._V[k,j], _ = self._rotation_cordic.run(self._V[k,i], self._V[k,j], theta)
    
    def do_one_round(self):
        for n in range(0, int(self._N/2)):
            a = self._round_state[n]
            b = self._round_state[int(n+(self._N/2)) % self._N]
            p = np.min([a,b])
            q = np.max([a,b])
            print("p = {p}, q = {q}".format(p=p, q=q))
            self.update_matrices(p,q)
        
        
        state_1 = self._round_state[1]
        
        for n in range(1,self._N-1):
            self._round_state[n] = self._round_state[n+1]
        self._round_state[self._N-1] = state_1
        
        print(self._round_state)
    
    def do_one_sweep(self):
        self.init_round_state()
        for i in range(self._N-1):
            self.do_one_round()
    
    def run(self, A):
        self.get_A(A)
        self.init_V()
        for i in range(self._n_sweeps):
            self.do_one_sweep()
        return self._A, self._V
        
        
    
            

In [44]:
n_iter = 12
n_word = 18
n_frac = 15
is_signed = True
N = 4
n_sweeps = 2
jacobi = Jacobi(n_cordic_iterations = n_iter, n_word = n_word, n_frac = n_frac, is_signed = is_signed, matrix_dim = N,
                n_sweeps = n_sweeps)


In [45]:
np.random.seed(17)
n_word_in = 16
A = (np.random.rand(N,N)*2)-1
A = (A + A.T)/2
A_cp = np.copy(A)
A = Fxp(A, n_word = n_word_in, n_frac = n_frac, signed = is_signed, rounding = "around")
A

fxp-s16/15([[-0.41067505  0.31756592 -0.76940918 -0.0680542 ]
            [ 0.31756592  0.31265259 -0.00466919  0.45288086]
            [-0.76940918 -0.00466919  0.89135742 -0.88876343]
            [-0.0680542   0.45288086 -0.88876343  0.30484009]])

In [46]:
jacobi.get_A(A)
jacobi.init_V()
jacobi.update_matrices(1,2)

In [47]:
A_out, W_out = jacobi.run(A)

p = 0, q = 2
p = 1, q = 3
[0 2 3 1]
p = 0, q = 3
p = 1, q = 2
[0 3 1 2]
p = 0, q = 1
p = 2, q = 3
[0 1 2 3]
p = 0, q = 2
p = 1, q = 3
[0 2 3 1]
p = 0, q = 3
p = 1, q = 2
[0 3 1 2]
p = 0, q = 1
p = 2, q = 3
[0 1 2 3]


In [48]:
np.set_printoptions(suppress=True)
print(A_out)
print(W_out)

[[-1.09124756  0.          0.00125122  0.        ]
 [ 0.         -0.01687622  0.         -0.00125122]
 [ 0.00125122  0.          0.42657471  0.        ]
 [ 0.         -0.00125122  0.          1.78079224]]
[[ 0.709198    0.63635254 -0.07077026  0.29678345]
 [-0.30038452  0.32748413  0.86831665  0.22247314]
 [ 0.47781372 -0.12960815  0.41046143 -0.7663269 ]
 [ 0.42355347 -0.68695068  0.27099609  0.5255127 ]]


In [49]:
w,v = np.linalg.eig(A_cp)
print("Eigenvalues: ")
print(w)
print("Eigenvectors: ")
print(v)

Eigenvalues: 
[ 1.77959787 -1.09085801 -0.01678166  0.42624246]
Eigenvectors: 
[[ 0.28751496  0.71204724 -0.63670741 -0.07019656]
 [ 0.22551769 -0.29822757 -0.32735224  0.86778027]
 [-0.77145222  0.46836782  0.13016474  0.41054867]
 [ 0.52090152  0.42974485  0.685931    0.27107127]]


In [50]:
def sort_eigenvectors(v):
    x = v[:,v[0,:].argsort()]
    return x

In [51]:
real = sort_eigenvectors(np.abs(v))
calculated = sort_eigenvectors(np.abs(W_out))
print(real)
print(calculated)
err = np.array(real-calculated)
print(err)
print(np.sum(np.abs(err))/N)

[[0.07019656 0.28751496 0.63670741 0.71204724]
 [0.86778027 0.22551769 0.32735224 0.29822757]
 [0.41054867 0.77145222 0.13016474 0.46836782]
 [0.27107127 0.52090152 0.685931   0.42974485]]
[[0.07077026 0.29678345 0.63635254 0.709198  ]
 [0.86831665 0.22247314 0.32748413 0.30038452]
 [0.41046143 0.7663269  0.12960815 0.47781372]
 [0.27099609 0.5255127  0.68695068 0.42355347]]
[[-0.0005737053818044568 -0.009268484192489934 0.0003548669581704411
  0.00284923938313697]
 [-0.0005363773711719588 0.0030445430511147342 -0.00013189224264203991
  -0.002156953646657467]
 [8.724026608153412e-05 0.005125310783192671 0.0005565840060461902
  -0.009445902042036936]
 [7.517689532116067e-05 -0.004611170603065262 -0.001019687141792791
  0.006191382959980007]]
0.011507129231176139


In [ ]:
0.0006639494933603773
0.0017322439211510943
0.0006101127413154078